# testing pyhmmer work

In [2]:
# dependecies
import os
import pyhmmer
import tempfile

# database
import duckdb as ddb

# 
import learn2therm.utils

In [22]:
PATH = os.path.abspath("/Users/humoodalanzi/pfam/proteins copy/*.parquet")

'/Users/humoodalanzi/pfam/proteins copy/*.parquet'

## testing ground for loading funciton

In [3]:
# stolen from Evan's t1.0 scripts
with tempfile.TemporaryDirectory(dir='./') as tmpdir:
    # Establishing a connection with Duck DB
    conn = ddb.connect(tmpdir+'proteins.db', read_only=False)
    # Making a SQL table
    conn.execute("CREATE TABLE proteins AS SELECT * FROM read_parquet('../data/uniprot_chunk_0.parquet')")
    # Committing DB
    conn.commit()

     # get some test proteins to run resource test alignment on
    # considering the max protein length
    query_proteins = conn.execute(
        f"""SELECT pid, protein_seq, LENGTH(protein_seq) AS len FROM proteins 
        WHERE len<=250
        ORDER BY RANDOM()
        LIMIT 1000
        """).df()
    subject_proteins = conn.execute(
        f"""SELECT pid, protein_seq, LENGTH(protein_seq) AS len FROM proteins 
        WHERE len<=250
        ORDER BY RANDOM()
        LIMIT 1000
        """).df()

    # get some metadata about taxa protein join
    # protein total counts per organism was tracked in s0.3
    # but lets recompute that data considering a max protien length
    protein_counts = conn.execute(
        f"""SELECT taxid, COUNT(*) AS n_proteins
        FROM proteins
        WHERE LENGTH(protein_seq)<=250
        GROUP BY taxid""").df()

In [4]:
query_proteins

,pid,protein_seq,len
0,I3R7V8,MPSEKEKMLAGELYDASDPELVMERKRARRLTRRFNASDETESLRR...,185
1,L0JM53,MIGSDVVVAGSALAIAVAVGLVAHEWSHAAVLRLARVEYAVSYFPG...,149
2,I3R9Q8,MYEHILVPTDGSETAEYAVDQAVDIASKYGSTVHALYVIDVDATSY...,154
3,L0JY84,MKDVKLDPSEESTYECFDCGTVVVSATAPGRCPSCGADMRNRATPLE,47
4,I3R1G7,MANGKYAARKLKKDRQKRRWSDSEYARRERGLGKKSDPLEGAPQGR...,142
...,...,...,...
995,A0A1L3Q032,MKYVKEFVKLAKSDLKSSVVLYENKCYPQSIFFFAQSVEKANKALA...,193
996,D2RQ33,MRKSGTWMTIWDDRILETLRKEGGKPVGELAEEDGIRISHSSVSRR...,103
997,A0A0A7GGQ8,MPLFLTRKAIEEVDVGDVIEVHADDPSARKDIPEWAERAGHKVLSV...,61
998,A0A650CYC0,MPTIHLSIPEGMYEELRKKAEDMGIQITDLVKFYIRQGMEKEEEGE...,101


In [5]:
subject_proteins

,pid,protein_seq,len
0,A0A1I0FQE4,MDVTQIGLGATLLVIGTLTLIGPATLATGPLAYLVTGSTLVVTVAA...,58
1,A0A1I0M298,MGFGSYDESEQQEQTTSDEDVEAVNVHENDHEGKLSFESDLSTDEL...,60
2,A0A0Q2QPZ4,MYKIKDEWGEFLVRLARRAIEEYVRNGRTIKPPEDTPPQLWERMGV...,204
3,A0A0F8FH29,MTRFIKYHPRSNTYVIEKRAFLEEDLTLDGNVIVGPEVKFWKNLTV...,142
4,B8D531,MNRVYVTVLATLLIIVGLLLGASMWLDILRANTYVDTGELDWEIVE...,225
...,...,...,...
995,A0A1H9YYW1,MNNIEEMIQKAVELQANGLVTGQIANELNVSRETVTWLLTRSKKDV...,203
996,L0JKK2,MGTHVLVPLDGSSQAWAAFDHAVSNHDGGRITTLHVVDPMAGVYSD...,146
997,L0JZ77,MCVSSRMPITVVQLTLSALAAVFAFITAVYGRLNYADGELDRQKKI...,195
998,L0JZQ8,MTVFALLRVTPITDDDATEDVAAAIDALEEYDVEYETTPLATTLEA...,104


So, in theory, I could sample the .parquet files and create a df out of them for loading

In [ ]:
# goal to make this function
def load_protein_data(database, **kwargs):
    """
    Load the protein sequences and associated taxonomic information from the input CSV files
    for DB version 1.0, or from the input Parquet files for DB version 1.1.

    Parameters
    ----------
    database : _type_
        _description_
    """
    # TODO
    pass

## Putting the pieces

In [3]:
# stolen from Evan's t1.0 scripts
with tempfile.TemporaryDirectory(dir='./') as tmpdir:
    # Establishing a connection with Duck DB
    conn = ddb.connect(tmpdir+'proteins.db', read_only=False)
    # Making a SQL table
    conn.execute(f"CREATE TABLE proteins AS SELECT * FROM read_parquet('../data/uniprot_chunk_0.parquet')")
    # Committing DB
    conn.commit()

    # get some test proteins to run resource test alignment on
    # considering the max protein length
    query_proteins = conn.execute(
        """SELECT pid, protein_seq, LENGTH(protein_seq) AS len FROM proteins 
        WHERE len<=250
        ORDER BY RANDOM()
        LIMIT 1000
        """).df()

    # get some metadata about taxa protein join
    # protein total counts per organism was tracked in s0.3
    # but lets recompute that data considering a max protien length
    protein_counts = conn.execute(
        """SELECT taxid, COUNT(*) AS n_proteins
        FROM proteins
        WHERE LENGTH(protein_seq)<=250
        GROUP BY taxid""").df()

The lesson here is that I shouldn't do maby parquets w/o Hyak. Need to figure out to use vscode with Hyak.

In [4]:
def prefetch_targets(hmms_path: str):
    """
    Prefetch HMM profiles from a given HMM database.
    Parameters
    ----------
    hmms_path : str
        Path to the HMM database.
    Returns
    -------
    targets : pyhmmer.plan7.OptimizedProfileBlock
        The HMM profiles loaded from the database.
    """
    # amino acid alphabet and prefetched inputs
    amino_acids = pyhmmer.easel.Alphabet.amino()
    optimized_profiles = list(pyhmmer.plan7.HMMPressedFile(hmms_path))
    targets = pyhmmer.plan7.OptimizedProfileBlock(
        amino_acids, optimized_profiles)
    return targets

In [ ]:
def save_sequences_to_fasta(
        sequences: pd.core.frame.DataFrame,
        inputname: str = "input"):
    """
    Returns a list of SeqRecord objects and creates a corresponding input Fasta of them
    Parameters:
    ------------
    sequences : pandas.core.frame.DataFrame
        a dataframe with string amino acid sequences in a 'protein_seq' column
    input name : str, default = 'input'
        a name for the input fasta file
    Returns:
    ------------
    file : TextIOWrapper
        the input fasta file created from the list of SeqRecord objects
    Raises
    -------
    ValueError :
        if the input dataframe is empty
    AttributeError :
        if any of the sequences are invalid
    """
    # ensure input file has .fasta extension
    if not inputname.endswith('.fasta'):
        inputname = f"{os.path.splitext(inputname)[0]}.fasta"

    # check if input is empty
    if sequences.empty:
        raise ValueError("Input dataframe is empty")

    # check if sequences are valid
    for seq in sequences['protein_seq']:
        try:
            Seq(seq)
        except BaseException as exc:
            raise AttributeError("Invalid sequence") from exc

    # function
    records = []
    for index, seq in sequences.itertuples():
        try:
            record = SeqRecord(Seq(seq), id=str(index))
            records.append(record)
        except AttributeError as exc:
            raise AttributeError(f"Invalid sequence: {seq}") from exc

    # raise error if seq not valid
    if not records:
        raise AttributeError("No valid sequences found in input")

    with open(inputname, "w", encoding="utf-8") as file:
        SeqIO.write(records, file, "fasta")
    return file